In [1]:
import pandas as pd
import numpy as np
import scipy.stats as ss
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [2]:
portfolio = pd.read_excel(io="Workshop Data.xlsx",sheet_name="Portfolio")
TenYearYield = pd.read_excel(io="Indexes and Spreads Data 01.09.xlsx",sheet_name="10yUST Yields")
HYyield = pd.read_excel(io="Indexes and Spreads Data 01.09.xlsx",sheet_name="HY Index")
IGyield = pd.read_excel(io="Indexes and Spreads Data 01.09.xlsx",sheet_name="IG Index")
HYGyas = pd.read_excel(io="Indexes and Spreads Data 01.09.xlsx",sheet_name="HYG")
MoveIdx = pd.read_excel(io="MOVE Vix prices.xlsx",sheet_name="MOVE Index")
Vix = pd.read_excel(io="MOVE Vix prices.xlsx",sheet_name="VIX")
SPY_Idx = pd.read_excel(io="SP500 Index data.xlsx",sheet_name="SPY")

portfolio = portfolio.set_index("Date").sort_index()
TenYearYield = TenYearYield.set_index("Date").sort_index()
HYyield = HYyield.set_index("Date").sort_index()
IGyield = IGyield.set_index("Date").sort_index()
MoveIdx = MoveIdx.set_index("Date").sort_index()
Vix = Vix.set_index("Date").sort_index()
SPY_Idx = SPY_Idx.set_index("Date").sort_index()
HYGyas = HYGyas.set_index("Date").sort_index()

PortfCorrData = pd.DataFrame(portfolio.loc["2021-01-11":"2025-12-31","Total Value"]).merge(HYyield.loc["2021-01-11":"2025-12-31","OAS_SOVEREIGN_CURVE"],left_index=True,right_index=True)

PortfCorrData = PortfCorrData.loc["2021-01-11":"2025-12-31"].merge(HYyield.loc["2021-01-11":"2025-12-31","INDEX_Z_SPREAD_BP"],left_index=True,right_index=True)
PortfCorrData = PortfCorrData.rename(columns={"OAS_SOVEREIGN_CURVE":"HY_Index_OAS","INDEX_Z_SPREAD_BP":"HY_INDEX_Z_SPREAD"})

PortfCorrData = PortfCorrData.loc["2021-01-11":"2025-12-31"].merge(TenYearYield.loc["2021-01-11":"2025-12-31","PX_LAST"],left_index=True,right_index=True)
PortfCorrData = PortfCorrData.rename(columns={"PX_LAST":"10y_UST_INDEX_PX"})

PortfCorrData = PortfCorrData.loc["2021-01-11":"2025-12-31"].merge(IGyield.loc["2021-01-11":"2025-12-31","INDEX_Z_SPREAD_BP"],left_index=True,right_index=True)
PortfCorrData = PortfCorrData.loc["2021-01-11":"2025-12-31"].merge(IGyield.loc["2021-01-11":"2025-12-31","OAS_SOVEREIGN_CURVE"],left_index=True,right_index=True)
PortfCorrData = PortfCorrData.rename(columns={"INDEX_Z_SPREAD_BP":"IG_INDEX_Z_Spread","OAS_SOVEREIGN_CURVE":"IG_Index_OAS"})

PortfCorrData = PortfCorrData.loc["2021-01-11":"2025-12-31"].merge(SPY_Idx.loc["2021-01-11":"2025-12-31","PX_LAST"],left_index=True,right_index=True)
PortfCorrData = PortfCorrData.rename(columns={"PX_LAST":"SPX_LAST_PX"})

PortfCorrData = PortfCorrData.loc["2021-01-11":"2025-12-31"].merge(HYGyas.loc["2021-01-11":"2025-12-31","YAS_MOD_DUR"],left_index=True,right_index=True)
PortfCorrData = PortfCorrData.rename(columns={"YAS_MOD_DUR":"HYG_MOD_DUR_YAS"})

PortfCorrData = PortfCorrData.loc["2021-01-11":"2025-12-31"].merge(MoveIdx.loc["2021-01-11":"2025-12-31","PX_LAST"],left_index=True,right_index=True)
PortfCorrData = PortfCorrData.rename(columns={"PX_LAST":"MOVE Idx"})

PortfCorrData = PortfCorrData.loc["2021-01-11":"2025-12-31"].merge(Vix.loc["2021-01-11":"2025-12-31","PX_LAST"],left_index=True,right_index=True)
PortfCorrData = PortfCorrData.rename(columns={"PX_LAST":"Vix Idx"})

PortfCorrData=PortfCorrData.dropna()
PortfCorrData = PortfCorrData.sort_index()

In [3]:
monthportdata = PortfCorrData.copy()
monthportdata=monthportdata.resample("ME").first()

In [4]:
monthportdata.loc[:,"Total Returns"] = monthportdata["Total Value"].pct_change()
monthportdata.loc[:,"HY_Index_OAS_Diff"]=monthportdata["HY_Index_OAS"].diff()
monthportdata.loc[:,"HY_INDEX_Z_SPREAD_Diff"]=monthportdata["HY_INDEX_Z_SPREAD"].diff()
monthportdata.loc[:,"10yYieldDiff"] = monthportdata["10y_UST_INDEX_PX"].diff()
monthportdata.loc[:,"IG_Index_Z_Spread_Diff"]=monthportdata["IG_INDEX_Z_Spread"].diff()
monthportdata.loc[:,"IG_Index_OAS_Diff"]=monthportdata["IG_Index_OAS"].diff()
monthportdata.loc[:,"ReturnSPX_Idx"] = monthportdata["SPX_LAST_PX"].pct_change()

monthportdata=monthportdata.dropna()
monthportdata = monthportdata.sort_index(ascending=False)

In [5]:
#Z-Score adjustment Process

X_Std = StandardScaler().fit_transform(monthportdata[["HY_Index_OAS_Diff","10yYieldDiff","IG_Index_OAS_Diff","ReturnSPX_Idx"]])
pcav1 = PCA()
pcav1.fit(X_Std)

print(pcav1.explained_variance_ratio_)
print(pd.DataFrame(pcav1.components_, columns=["HY_Index_OAS_Diff","10yYieldDiff","IG_Index_OAS_Spread_Diff","ReturnSPX_Idx"]))

X_pca = pcav1.transform(X_Std)

monthportdata.loc[:,"PC1"]=X_pca[:,0]
monthportdata.loc[:,"PC2"]=X_pca[:,1]
monthportdata.loc[:,"PC3"]=X_pca[:,2]
monthportdata.loc[:,"PC4"]=X_pca[:,3]
#PortfCorrData.loc[:,"PC5"]=X_pca[:,4]

X = sm.add_constant(monthportdata[["PC1","PC2","PC3"]])
Y = monthportdata["Total Returns"]

model = sm.OLS(Y,X).fit()
model.summary()

[0.6463158  0.25597316 0.05922496 0.03848608]
   HY_Index_OAS_Diff  10yYieldDiff  IG_Index_OAS_Spread_Diff  ReturnSPX_Idx
0          -0.556741     -0.203105                 -0.565744       0.573342
1          -0.320483      0.924744                 -0.154075      -0.135648
2          -0.193031      0.148220                  0.754474       0.609540
3           0.741666      0.285706                 -0.294896       0.530414


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          Total Returns   R-squared:                       0.920
Model:                            OLS   Adj. R-squared:                  0.916
Method:                 Least Squares   F-statistic:                     211.7
Date:                Fri, 16 Jan 2026   Prob (F-statistic):           3.55e-30
Time:                        23:01:27   Log-Likelihood:                 216.94
No. Observations:                  59   AIC:                            -425.9
Df Residuals:                      55   BIC:                            -417.6
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0008      0.001     -0.927      0.358      -0.002       0.001
PC1            0.0076      0.001     14.831      0.000       0.007       0.009
PC2           -0.0155      0.001    -18.988      0.000      -0.017      -0.014
PC3           -0.0125      0.002     -7.392      0.000      -0.016      -0.009
==============================================================================
Omnibus:                        4.602   Durbin-Watson:                   2.351
Prob(Omnibus):                  0.100   Jarque-Bera (JB):                4.149
Skew:                           0.650   Prob(JB):                        0.126
Kurtosis:                       3.023   Cond. No.                         3.30
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [6]:
#Z-Score adjustment Process

X_Std = StandardScaler().fit_transform(monthportdata[["HY_Index_OAS_Diff","10yYieldDiff","IG_Index_OAS_Diff","ReturnSPX_Idx"]])
pcav1 = PCA()
pcav1.fit(X_Std)

print(pcav1.explained_variance_ratio_)
print(pd.DataFrame(pcav1.components_, columns=["HY_Index_OAS_Diff","10yYieldDiff","IG_Index_OAS_Spread_Diff","ReturnSPX_Idx"]))

X_pca = pcav1.transform(X_Std)

monthportdata.loc[:,"PC1"]=X_pca[:,0]
monthportdata.loc[:,"PC2"]=X_pca[:,1]
monthportdata.loc[:,"PC3"]=X_pca[:,2]
monthportdata.loc[:,"PC4"]=X_pca[:,3]
#PortfCorrData.loc[:,"PC5"]=X_pca[:,4]

X = sm.add_constant(monthportdata[["PC1","PC2"]])
Y = monthportdata["Total Returns"]

model = sm.OLS(Y,X).fit()
model.summary()

[0.6463158  0.25597316 0.05922496 0.03848608]
   HY_Index_OAS_Diff  10yYieldDiff  IG_Index_OAS_Spread_Diff  ReturnSPX_Idx
0          -0.556741     -0.203105                 -0.565744       0.573342
1          -0.320483      0.924744                 -0.154075      -0.135648
2          -0.193031      0.148220                  0.754474       0.609540
3           0.741666      0.285706                 -0.294896       0.530414


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          Total Returns   R-squared:                       0.841
Model:                            OLS   Adj. R-squared:                  0.835
Method:                 Least Squares   F-statistic:                     148.2
Date:                Fri, 16 Jan 2026   Prob (F-statistic):           4.26e-23
Time:                        23:01:29   Log-Likelihood:                 196.59
No. Observations:                  59   AIC:                            -387.2
Df Residuals:                      56   BIC:                            -380.9
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0008      0.001     -0.662      0.510      -0.003       0.002
PC1            0.0076      0.001     10.599      0.000       0.006       0.009
PC2           -0.0155      0.001    -13.570      0.000      -0.018      -0.013
==============================================================================
Omnibus:                        0.917   Durbin-Watson:                   1.999
Prob(Omnibus):                  0.632   Jarque-Bera (JB):                0.888
Skew:                           0.095   Prob(JB):                        0.641
Kurtosis:                       2.430   Cond. No.                         1.61
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""